In [ ]:
# Dependencies
import tweepy
import json
import numpy as np
from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
import time

import plotly
plotly.tools.set_credentials_file(username='joelboyette', api_key='HIyPnjcR2BP793dZBmRC')

import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

In [ ]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True)

In [ ]:
#list to hold tweet data
tweet_text = []
tweet_times = []
unique_ids = []
tweet_user = []
tweet_handle = []
tweet_followers = []

tweet_search_keyword = []
tweet_post_retweets = []
tweet_likes = []
bad_handle = []

oldest_tweet = None
sinceId = None
tweet_count = 0


In [ ]:
top_stocks = pd.read_csv("SP_100_Tickers_final.csv", encoding ='latin')

#clean out bad characters
top_stocks["Twitter_handle"] =  top_stocks.Twitter_handle.str.replace('\xa0','')

top_stocks_handle = top_stocks["Twitter_handle"].tolist()

stock_keywords = top_stocks_handle


In [ ]:
#loop through keywords
for keyword in stock_keywords:

    oldest_tweet = None
    
    #time.sleep(5)
    
    try:
        for x in range(1,12):
            #put tweeter searches in list
            public_tweets = api.user_timeline(keyword, count = 200, max_id = oldest_tweet)

            # Loop through all tweets
            for tweet in public_tweets:

                tweet_id = tweet["id"]

                 # if Id is unique tweet data will be added
                if tweet_id not in unique_ids:
                    
                    unique_ids.append(tweet_id)
                    tweet_search_keyword.append(keyword)
                    
                    fix_string = tweet["text"].rstrip('\r\n')
                    tweet_text.append(fix_string)
                    
                    tweet_user.append(tweet["user"]["name"])
                    
                    tweet_followers.append(tweet["user"]["followers_count"])
                    
                    tweet_times.append(tweet["created_at"])
                    
                    tweet_post_retweets.append(tweet["retweet_count"])
                    tweet_likes.append(tweet["favorite_count"])
                        
                    oldest_tweet = tweet_id - 1
                    tweet_count += 1

        
        print(keyword + " grabbed num tweets: " + str(tweet_count))
    
    except:
        bad_handle.append(keyword)
        print(f'{keyword} doesnt have twitter')


In [ ]:
# Add each datetime object into the array
tweet_time_objects = []
for x in range(len(tweet_times)):
    tweet_datetime = datetime.strptime(tweet_times[x], "%a %b %d %H:%M:%S %z %Y")
    tweet_time_objects.append(tweet_datetime)


In [ ]:
#check to make sure all list are the same length
print(len(tweet_search_keyword))
print(len(tweet_user))

print(len(tweet_followers))

print(len(unique_ids))
print("---------")
print(len(tweet_text))
print(len(tweet_time_objects))
print(bad_handle)

In [ ]:
tweet_data.drop_duplicates(keep=False)

tweet_data.dropna(how="any")
tweet_data = tweet_data.loc(tweet_data["Name"] != "Joel")


In [ ]:
tweet_data = pd.DataFrame({"Twitter_handle": tweet_search_keyword,
                            'Name':tweet_user,
                           'Followers':tweet_followers,
                            'Id':unique_ids,
                            'Text': tweet_text,
                           'Time Stamp':tweet_time_objects,
                            'Time Retweeted':tweet_post_retweets,
                            'Times liked':tweet_likes })

tweet_data_all = pd.merge(tweet_data,top_stocks[["Twitter_handle","industry","sector","Symbol"]], 
                on="Twitter_handle", how = "left")

In [ ]:
# Identify incomplete rows
#df.count()
# Drop all rows with missing information
#df = df.dropna(how='any')

In [ ]:
counter = 0

for index, row in tweet_data_all.iterrows():
    
    re_retweet = re.match(r'^(RT @)',row["Text"])
    re_url = re.findall(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',row["Text"])
    re_hash = re.findall(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)',row["Text"])
    re_at = re.findall(r'(^|[^\w@/\!?=&])@(\w{1,15})\b',row["Text"])

    if re_retweet:
        tweet_data_all.loc[index,"Retweet"] = 1
    else:
        tweet_data_all.loc[index,"Retweet"] = 0
        
    if re_url:
        tweet_data_all.lo c[index,"URL"] = 1
    else:
        tweet_data_all.loc[index,"URL"] = 0
        
    if re_hash:
        tweet_data_all.loc[index,"Hashtag"] = 1
    else:
        tweet_data_all.loc[index,"Hashtag"] = 0
        
    if re_at and not re_retweet:
        tweet_data_all.loc[index,"@ Symbol"] = 1
    else:
        tweet_data_all.loc[index,"@ Symbol"] = 0
    
    #if not re_at and not re_retweet and not re_hash and not re_url aad not re_retweet:
    #    tweet_data_all.loc[index,"Other Tweet"] = 1
    #else:
    #    tweet_data_all.loc[index,"Other Tweet"] = 0
    
    counter += 1
        
    # Print tweets for preview
    if counter % 100 == 0:
        print(f'rows processed {counter}')
    
tweet_data_all = tweet_data_all.dropna(how='any')
tweet_data_all.to_csv("tweet_data_all.csv")

In [ ]:
industry_group = tweet_data_all.groupby("industry")

industry_summary = pd.DataFrame(industry_group["Text"].count())
industry_summary = industry_summary.rename(columns={"Text" : "Tweet Count"})

industry_summary.sort_values(by="Tweet Count", ascending= False)
industry_summary = industry_summary.reset_index()

industry_summary.head(5)


In [ ]:
company_industry_stats = tweet_data_all[["Name","industry"]]
company_industry_stats = company_data.drop_duplicates(['industry','Name'])

company_industry_df = company_industry_stats.groupby("industry",as_index=False).count()
company_industry_df = company_industry_df.rename(columns = {"Name" : "Num of Companies"})

company_industry_df["Percent of Total"] = (company_industry_df['Num of Companies'] / company_industry_df["industry"].count() ) 

industry_final = pd.merge(industry_summary,company_industry_df[["industry","Num of Companies","Percent of Total"]], on="industry" , how = "left")

industry_final["Percent of Total"] = industry_final["Percent of Total"].map('{:,.2%}'.format)

industry_final.head(5)


In [ ]:
sector_list = industry_final["industry"].tolist()
count_list =  industry_final["Tweet Count"].tolist()
percent_list = industry_final["Percent of Total"].tolist()

data = [go.Bar(x=sector_list,y=count_list, text = percent_list, textposition = 'auto',
            marker=dict(color='rgb(58,200,225)',line=dict(color='rgb(8,48,107)',width=1.5),),opacity=0.6)]

layout = go.Layout(title='Industry Breakdown of Tweets',xaxis=dict(tickangle=-65),
                   yaxis=dict(title="Tweets",showgrid=False),margin=go.Margin(b=200))

fig = go.Figure(data=data, layout=layout)



py.image.save_as(fig, filename='industry_breakdown_tweets.png')

py.iplot(fig, filename='basic-bar')


In [ ]:
company_group = tweet_data_all.groupby("Name")
company_summary = pd.DataFrame(company_group["Text"].count())
company_summary["Retweet"] = company_group["Retweet"].sum()
company_summary["URL"] = company_group["URL"].sum()
company_summary["Hashtag"] = company_group["Hashtag"].sum()
company_summary["@ Symbol"] = company_group["@ Symbol"].sum()

company_summary = company_summary.rename(columns={"Text" : "Tweet Count"})
company_summary = company_summary.reset_index()
company_summary.head()


In [ ]:
name_list = company_summary["Name"].tolist()
retweet_list = company_summary["Retweet"].tolist()
url_list = company_summary["URL"].tolist()
hashtag_list = company_summary["Hashtag"].tolist()
symbol_list = company_summary["@ Symbol"].tolist()

trace1 = go.Bar(x=name_list, y=retweet_list, name="retweet",marker=dict(color = 'rgb(58,200,225)'))
trace2 = go.Bar(x=name_list, y=url_list, name='url',marker=dict(color = 'rgb(113, 165, 214)'))
trace3 = go.Bar(x=name_list, y=symbol_list, name='@ Symbol',marker=dict(color = 'rgb(113, 214, 140)'))
trace4 = go.Bar(x=name_list, y=hashtag_list, name='Hashtag',marker=dict(color = 'rgb(214, 113, 143)'))

data = [trace1, trace2, trace3, trace4] 
layout = go.Layout( title='Types of Tweets by Company', xaxis=dict(tickangle=-85,dtick = 1.5), 
                   yaxis=dict(title="Type of Tweet",showgrid=False) , barmode='stack', margin=go.Margin(b=160))

fig = go.Figure(data=data, layout=layout)

py.image.save_as(fig, filename='tweet_type_company.png')

py.iplot(fig, filename='stacked-bar')